In [1]:
nombre = "Ricardo Madrigal Urencio"
print(nombre)

Ricardo Madrigal Urencio


In [2]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace
from pyspark.sql.functions import date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [3]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/ecobici/2019"

In [4]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)
df.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: string (nullable = true)
 |-- Bici: string (nullable = true)
 |-- Ciclo_Estacion_Retiro: string (nullable = true)
 |-- Fecha_Retiro: string (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: string (nullable = true)
 |-- Fecha_Arribo: string (nullable = true)
 |-- Hora_Arribo: string (nullable = true)
 |-- _c9: timestamp (nullable = true)



In [5]:
df.count()

7718156

In [6]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|    _c9|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+
|             0|           0|   0|                    0|           0|          0|                    0|           0|          0|7718155|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+



In [7]:
df.show(20, truncate = False)

+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+----+
|Genero_Usuario|Edad_Usuario|Bici |Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|_c9 |
+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+----+
|M             |34          |8861 |143                  |01/03/2019  |00:00:54   |68                   |01/03/2019  |00:13:55   |NULL|
|F             |33          |2601 |51                   |01/03/2019  |00:01:01   |53                   |01/03/2019  |00:07:16   |NULL|
|M             |25          |8257 |340                  |01/03/2019  |00:01:26   |324                  |01/03/2019  |00:09:26   |NULL|
|M             |32          |15007|139                  |01/03/2019  |00:01:42   |155                  |01/03/2019  |00:06:20   |NULL|
|F             |22          |7601 |81                  

In [8]:
df.filter("_c9 is not null").show(20, truncate = False)

+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|_c9                |
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+
|M             |44          |9944|267                  |06/03/2019  |08:06:23   |32:11.8              |10          |00:00:00   |2024-10-19 08:41:11|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+



In [9]:
datos = df\
.withColumn("Edad_Usuario", col("Edad_Usuario").cast(FloatType())) \
.withColumn("Bici", col("Bici").cast(IntegerType())) \
.withColumn("Ciclo_Estacion_Retiro", col("Ciclo_Estacion_Retiro").cast(IntegerType())) \
.withColumn("Fecha_Retiro", to_date(col("Fecha_Retiro"), "dd/MM/yyyy")) \
.withColumn("Hora_Retiro", date_format(col("Hora_Retiro"), "HH:mm:ss")) \
.withColumn("Ciclo_Estacion_Arribo", col("Ciclo_Estacion_Arribo").cast(IntegerType())) \
.withColumn("Fecha_Arribo", to_date(col("Fecha_Arribo"), "dd/MM/yyyy")) \
.withColumn("Hora_Arribo", date_format(col("Hora_Arribo"), "HH:mm:ss"))

In [10]:
datos.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: string (nullable = true)
 |-- _c9: timestamp (nullable = true)



In [11]:
datos = datos.select(['Genero_Usuario', 'Edad_Usuario', 'Bici', 
                      'Ciclo_Estacion_Retiro', 'Fecha_Retiro', 'Hora_Retiro', 
                     'Ciclo_Estacion_Arribo', 'Fecha_Arribo', 'Hora_Arribo'])

In [12]:
datos.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: string (nullable = true)



In [13]:
datos.orderBy('Hora_Retiro', ascending = False).show(2000, truncate = False)

+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+
|Genero_Usuario|Edad_Usuario|Bici |Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|
+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+
|M             |27.0        |10619|258                  |2019-04-05  |23:59:59   |120                  |2019-04-06  |00:15:32   |
|M             |33.0        |11801|27                   |2019-12-07  |23:59:59   |259                  |2019-12-08  |00:08:19   |
|M             |38.0        |8414 |158                  |2019-03-16  |23:59:59   |141                  |2019-03-17  |00:04:26   |
|M             |51.0        |12494|248                  |2019-05-08  |23:59:59   |248                  |2019-05-09  |00:08:51   |
|M             |57.0        |7261 |43                   |2019-03-26  |23:59:59   |123     

In [14]:
#Separar Fechas y Horas de Retiro en Campos Individuales

from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfr = "Fecha_Retiro"
colhr = "Hora_Retiro"

datos_tiempo_retiro = datos.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Retiro"),
    date_format(col(colfr), "EEEE").alias("dia_sem_retiro"),
    dayofmonth(col(colfr)).alias("num_dia_retiro"),
    month(col(colfr)).alias("num_mes_retiro"),
    date_format(col(colfr), "MMMM").alias("mes_retiro"),
    year(col(colfr)).alias("anio_retiro"),
    hour(col(colhr)).alias("hora_retiro"),
    minute(col(colhr)).alias("minuto_retiro"),    
    second(col(colhr)).alias("segundo_retiro")
    
)

datos_tiempo_retiro.printSchema()

datos_tiempo_retiro.orderBy('num_mes_retiro', ascending = True).show()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- dia_sem_retiro: string (nullable = true)
 |-- num_dia_retiro: integer (nullable = true)
 |-- num_mes_retiro: integer (nullable = true)
 |-- mes_retiro: string (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- hora_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)

+--------------+------------+-----+---------------------+--------------+--------------+--------------+----------+-----------+-----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario| Bici|Ciclo_Estacion_Retiro|dia_sem_retiro|num_dia_retiro|num_mes_retiro|mes_retiro|anio_retiro|hora_retiro|minuto_retiro|segundo_retiro|
+--------------+------------+-----+---------------------+--------------+--------------+--------------+----------+----

In [15]:
#Separar Fechas y Horas de Arribo en Campos Individuales

from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfa = "Fecha_Arribo"
colha = "Hora_Arribo"

datos_tiempo_arribo = datos.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Arribo"),
    date_format(col(colfa), "EEEE").alias("dia_sem_arribo"),
    dayofmonth(col(colfa)).alias("num_dia_arribo"),
    month(col(colfa)).alias("num_mes_arribo"),
    date_format(col(colfa), "MMMM").alias("mes_arribo"),
    year(col(colfa)).alias("anio_arribo"),
    hour(col(colha)).alias("hora_arribo"),
    minute(col(colha)).alias("minuto_arribo"),    
    second(col(colha)).alias("segundo_arribo")
)

datos_tiempo_arribo.printSchema()

datos_tiempo_arribo.orderBy('num_mes_arribo', ascending = True).show()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- dia_sem_arribo: string (nullable = true)
 |-- num_dia_arribo: integer (nullable = true)
 |-- num_mes_arribo: integer (nullable = true)
 |-- mes_arribo: string (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- hora_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)

+--------------+------------+-----+---------------------+--------------+--------------+--------------+----------+-----------+-----------+-------------+--------------+
|Genero_Usuario|Edad_Usuario| Bici|Ciclo_Estacion_Arribo|dia_sem_arribo|num_dia_arribo|num_mes_arribo|mes_arribo|anio_arribo|hora_arribo|minuto_arribo|segundo_arribo|
+--------------+------------+-----+---------------------+--------------+--------------+--------------+----------+----

In [16]:
#Separar Fechas y Horas de Retiro y Arribo en Campos Individuales

from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs = "Fecha_Retiro"
colhs = "Hora_Retiro"
colfa = "Fecha_Arribo"
colha = "Hora_Arribo"

ecobici = datos.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Retiro"),
    date_format(col(colfs), "EEEE").alias("dia_sem_retiro"),
    dayofmonth(col(colfs)).alias("num_dia_retiro"),
    month(col(colfs)).alias("num_mes_retiro"),
    date_format(col(colfs), "MMMM").alias("mes_retiro"),
    year(col(colfs)).alias("anio_retiro"),
    hour(col(colhs)).alias("hora_retiro"),
    minute(col(colhs)).alias("minuto_retiro"),    
    second(col(colhs)).alias("segundo_retiro"),
    col("Ciclo_Estacion_Arribo"),
    date_format(col(colfa), "EEEE").alias("dia_sem_arribo"),
    dayofmonth(col(colfa)).alias("num_dia_arribo"),
    month(col(colfa)).alias("num_mes_arribo"),
    date_format(col(colfa), "MMMM").alias("mes_arribo"),
    year(col(colfa)).alias("anio_arribo"),
    hour(col(colha)).alias("hora_arribo"),
    minute(col(colha)).alias("minuto_arribo"),    
    second(col(colha)).alias("segundo_arribo")
    
)

ecobici.printSchema()

ecobici.orderBy('num_mes_retiro', ascending = True).show()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- dia_sem_retiro: string (nullable = true)
 |-- num_dia_retiro: integer (nullable = true)
 |-- num_mes_retiro: integer (nullable = true)
 |-- mes_retiro: string (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- hora_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- dia_sem_arribo: string (nullable = true)
 |-- num_dia_arribo: integer (nullable = true)
 |-- num_mes_arribo: integer (nullable = true)
 |-- mes_arribo: string (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- hora_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)

+--------------+------------+-----

In [17]:
ecobici.write.mode("overwrite").option("header", "true").csv("hdfs://namenode:9000/tmp/amd/ecobicilimpio")
print("Done")

Done
